In [ ]:
import torch
import itertools

def assign_matrices(N, Nr):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create indices for all possible combinations of receivers and senders
    indices = torch.tensor(list(itertools.product(range(N), range(N))), dtype=torch.long, device=device)

    # Remove combinations where receiver and sender are the same
    mask = indices[:, 0] != indices[:, 1]
    indices = indices[mask]

    # Initialize Rr and Rs tensors
    Rr = torch.zeros(N, Nr, device=device)
    Rs = torch.zeros(N, Nr, device=device)

    # Assign 1 to appropriate positions in Rr and Rs
    Rr[indices[:, 0], torch.arange(Nr)] = 1
    Rs[indices[:, 1], torch.arange(Nr)] = 1

    print("Rr:", Rr)
    print("Rs:", Rs)
    return Rr, Rs



# Example usage
# Rr, Rs = assign_matrices(5, 20)

In [ ]:
def assign_matrices_SV(N, Nt, Nv):
    # Create indices for all possible combinations of keys and values
    indices = torch.tensor(list(itertools.product(range(N), range(Nv))), dtype=torch.long)

    # Initialize Rk and Rv tensors
    Rk = torch.zeros(N, Nt)
    Rv = torch.zeros(Nv, Nt)

    # Assign 1 to appropriate positions in Rk and Rv
    Rk[indices[:, 0], torch.arange(Nt)] = 1
    Rv[indices[:, 1], torch.arange(Nt)] = 1

    # Move tensors to GPU if available
    if torch.cuda.is_available():
        Rk = Rk.cuda()
        Rv = Rv.cuda()

    return Rk,Rv

Rk, Rv = assign_matrices_SV(3, 6, 2)

print("Rk:", Rk)
# print(Rk @ Rv)
print("Rv:", Rv)



Rk: tensor([[1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1.]])
Rv: tensor([[1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 1.]])


In [ ]:
import torch
import itertools
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, BatchNorm1d
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


class IN(torch.nn.Module):
    def __init__(self, hidden=3, N_p=1000, params=60, N_v=1000, params_v=5, De=1, Do=6, softmax=False):
        super(IN, self).__init__()  # Call the superclass's __init__ method
        self.hidden = hidden
        self.Np = N_p  # N
        self.P = params
        self.Npp = self.Np * (self.Np - 1)  # Nr
        self.Nv = N_v
        self.S = params_v
        self.Npv = self.Np * self.Nv  # Nt
        self.De = De
        self.Do = Do
        self.softmax = softmax

        self.fr_pp = Seq(Lin(2 * self.P, self.hidden), ReLU(), Lin(self.hidden, self.De))
        self.fr_pv = Seq(Lin(self.P + self.S, self.hidden), ReLU(), Lin(self.hidden, self.De))
        self.fo = Seq(Lin(self.P + (2 * self.De), self.hidden), ReLU(), Lin(self.hidden, self.Do))
        self.fc = nn.Linear(self.Do, 1)

        self.Rr,self.Rs = assign_matrices(self.Np, self.Npp)
        self.Rk,self.Rv = assign_matrices(self.Np, self.Npv, self.Nv)


    def forward(self, x, y):
        # PF Candidate
        # Orr = self.tmul(x, Rr)
        # Ors = self.tmul(x, Rs)
        Orr = x @ self.Rr
        Ors = x @ self.Rr
        B = torch.cat([Orr, Ors], 1)

        # First MLP
        B = B.transpose(1, 2).contiguous()
        E = self.fr_pp(B.view(-1, 2 * self.P))
        del B

        E = E.transpose(1, 2).contiguous()
        # Ebar_pp = self.tmul(E, Rr.t().contiguous())
        Ebar_pp = (E @ self.Rr.t().contiguous())
        del E

        # Secondary Vertex
        # PF Candidate
        #Ork = self.tmul(x, self.Rk)
        #Orv = self.tmul(y, self.Rv)
        Ork = x @ self.Rk
        Orv = y @ self.Rv
        B = torch.cat([Ork, Orv], 1)

        # First MLP
        B = B.transpose(1, 2).contiguous()
        E = self.fr_pv(B.view(-1, self.S + self.P))
        del B
        E = E.transpose(1, 2).contiguous()

        #Ebar_pv = self.tmul(E, torch.transpose(self.Rk, 0, 1).contiguous())
        #Ebar_vp = self.tmul(E, torch.transpose(self.Rv, 0, 1).contiguous())
        #Ebar_pv = (E @ self.Rk.t().contiguous())
        #typo in text, see dimention of matrix
        Ebar_vp = (E @ self.Rk.t().contiguous())
        del E


        # Final output maxtrix for particels
        #C = torch.cat([x, Ebar_pp, Ebar_pv], 1)
        #del Ebar_pp
        #del Ebar_pv
        #C = torch.transpose(C, 1, 2).contiguous()
        C = torch.cat([x, Ebar_pp, Ebar_pv], 1).transpose(1, 2).contiguous()
        del Ebar_pp
        del Ebar_pv
        # Second MLP
        O = self.fo(C.view(self.P + (2 * self.De)))
        del C


        #Taking the sum of over each particle/vertex
        N = torch.sum(O, dim=1)
        del O


        #Classification MLP
        N = self.fc(N)

        if self.softmax:
            N = nn.Softmax(dim=-1)(N)


        return N







